The ipynb builds a model using the exising tsgm agent cgan

In [1]:
import tsgm 
import tensorflow as tf
from tensorflow import keras
import numpy as np

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("Memory growth enabled for GPU.")
    except RuntimeError as e:
        print(e)

memory_limit = 6000
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=memory_limit)]
        )
        print(f"Memory limit set to {memory_limit} MB for GPU.")
    except RuntimeError as e:
        print(e)

import pandas as pd 
import numpy as np
from tensorflow.keras.layers import Reshape



2024-08-30 01:58:29.804462: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-30 01:58:29.855202: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-30 01:58:29.855241: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-30 01:58:29.856933: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-30 01:58:29.866281: I tensorflow/core/platform/cpu_feature_guar

Physical devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized


In [7]:
latent_dim = 100
seq_len = 100
feature_dim = 2

dataset = np.load("/home/junze/.jupyter/Data transfer and loading/dataset_1.csv.npy")
print("Dataset shape:", dataset.shape)
seq_len, feature_dim = dataset.shape[1], dataset.shape[2]
latent_dim = 100

# Create a custom encoder
class Sampling(tf.keras.layers.Layer):
    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

def create_custom_encoder(seq_len, feature_dim, latent_dim):
    inputs = tf.keras.Input(shape=(seq_len, feature_dim))
    x = tf.keras.layers.Conv1D(32, 3, activation="relu", padding="same")(inputs)
    x = tf.keras.layers.Conv1D(64, 3, activation="relu", padding="same")(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(256, activation="relu")(x)
    
    z_mean = tf.keras.layers.Dense(latent_dim, name="z_mean")(x)
    z_log_var = tf.keras.layers.Dense(latent_dim, name="z_log_var")(x)
    z = Sampling()([z_mean, z_log_var])
    
    encoder = tf.keras.Model(inputs=inputs, outputs=[z_mean, z_log_var, z])
    return encoder

# Create a custom decoder
def create_custom_decoder(seq_len, feature_dim, latent_dim):
    latent_inputs = tf.keras.Input(shape=(latent_dim,))
    x = tf.keras.layers.Dense(seq_len * 64, activation="relu")(latent_inputs)
    x = tf.keras.layers.Reshape((seq_len, 64))(x)
    x = tf.keras.layers.Conv1D(32, 3, activation="relu", padding="same")(x)
    x = tf.keras.layers.Conv1D(16, 3, activation="relu", padding="same")(x)
    outputs = tf.keras.layers.Conv1D(feature_dim, 1, activation="sigmoid")(x)
    
    decoder = tf.keras.Model(inputs=latent_inputs, outputs=outputs)
    return decoder

encoder = create_custom_encoder(seq_len, feature_dim, latent_dim)
decoder = create_custom_decoder(seq_len, feature_dim, latent_dim)

# Create the BetaVAE
cvae = tsgm.models.cvae.BetaVAE(encoder, decoder)

print("Encoder summary:")
encoder.summary()

print("\nDecoder summary:")
decoder.summary()

# Compile and fit the model
cvae.compile(optimizer=tf.keras.optimizers.Adam())
cvae.fit(dataset, epochs=1000, batch_size=64)

Dataset shape: (500, 100, 2)
Encoder summary:
Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_18 (InputLayer)       [(None, 100, 2)]             0         []                            
                                                                                                  
 conv1d_32 (Conv1D)          (None, 100, 32)              224       ['input_18[0][0]']            
                                                                                                  
 conv1d_33 (Conv1D)          (None, 100, 64)              6208      ['conv1d_32[0][0]']           
                                                                                                  
 flatten_7 (Flatten)         (None, 6400)                 0         ['conv1d_33[0][0]']           
                                              

2024-08-30 02:48:39.114763: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-08-30 02:48:41.369855: I external/local_xla/xla/service/service.cc:168] XLA service 0x7fb595f61dd0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-08-30 02:48:41.369903: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4090, Compute Capability 8.9
2024-08-30 02:48:41.378544: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1724957321.525940 3144417 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


8/8 [==============================] - 8s 78ms/step - loss: 20964150741283.5547 - reconstruction_loss: 66968368.0000 - kl_loss: 23640364023808.0000
Epoch 2/1000
8/8 [==============================] - 0s 14ms/step - loss: 745876815872.0000 - reconstruction_loss: 66427692.0000 - kl_loss: 518904250368.0000
Epoch 3/1000
8/8 [==============================] - 0s 15ms/step - loss: 272636784.0000 - reconstruction_loss: 66356236.0000 - kl_loss: 92477104.0000
Epoch 4/1000
8/8 [==============================] - 0s 15ms/step - loss: 70722873.7778 - reconstruction_loss: 66352004.0000 - kl_loss: 2309291.5000
Epoch 5/1000
8/8 [==============================] - 0s 15ms/step - loss: 67513514.6667 - reconstruction_loss: 66353460.0000 - kl_loss: 112430.5156
Epoch 6/1000
8/8 [==============================] - 0s 14ms/step - loss: 67447869.3333 - reconstruction_loss: 66350268.0000 - kl_loss: 58744.1719
Epoch 7/1000
8/8 [==============================] - 0s 14ms/step - loss: 67399926.2222 - reconstruction_

: 

In [16]:
cvae.compile()
cvae.fit(dataset, epochs=1000, batch_size=64)

Epoch 1/1000


ValueError: in user code:

    File "/home/junze/anaconda3/envs/tsgm1/lib/python3.10/site-packages/keras-2.15.0-py3.10.egg/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/home/junze/anaconda3/envs/tsgm1/lib/python3.10/site-packages/keras-2.15.0-py3.10.egg/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/junze/anaconda3/envs/tsgm1/lib/python3.10/site-packages/keras-2.15.0-py3.10.egg/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/home/junze/anaconda3/envs/tsgm1/lib/python3.10/site-packages/tsgm-0.0.5-py3.10.egg/tsgm/models/cvae.py", line 79, in train_step
        z_mean, z_log_var, z = self.encoder(data)
    File "/home/junze/anaconda3/envs/tsgm1/lib/python3.10/site-packages/keras-2.15.0-py3.10.egg/keras/src/utils/traceback_utils.py", line 70, in error_han